# 🏥 Gemma-Health Sentinel
## نظام الطوارئ الذكي — Powered by MedGemma 4B

**MedGemma Impact Challenge 2026** — Agentic Workflow for Emergency Medicine

---

### Patient Flow: الاستقبال → الطوارئ → التشخيص المعمق

This notebook demonstrates an intelligent emergency department system that:
1. **Reception** — Patient registration + AI-generated medical summary
2. **Emergency** — Digital ER form with real-time drug interaction alerts
3. **Deep Diagnosis** — AI detective loop for complex case analysis

> ⚠️ This system is a clinical decision support tool — the physician always has final authority.

## 📦 Cell 1: Install Dependencies

In [15]:
!pip install -q transformers>=4.40.0 torch>=2.0.0 accelerate>=0.27.0 bitsandbytes>=0.43.0 gradio>=4.20.0 pandas>=2.0.0

## 🗄️ Cell 2: Initialize Database + Seed Demo Data

In [9]:
import sys, os
sys.path.insert(0, os.getcwd())

from db.init_db import init_database
from db.seed_data import seed_all
from db.queries import get_patient_full_record, get_all_patients_summary

# Initialize and seed
init_database()
seed_all()

# Verify
patients = get_all_patients_summary()
print(f"\n📊 Loaded {len(patients)} patients:")
for p in patients:
    print(f"   {p['patient_id']}. {p['name']} — {p['age']} سنة — {p['gender']}")

# Show Sarah's record (key demo patient)
print("\n🔍 سجل المريضة المحورية (سارة خالد):")
record = get_patient_full_record(3)
print(f"   Diseases: {[d['disease_name'] for d in record['chronic_diseases']]}")
print(f"   Abnormal labs: {[l['test_name'] for l in record['abnormal_labs']]}")

✅ Database initialized successfully at: h:\projectapp\medgemma\db\hospital.db
✅ Seed data inserted successfully — 5 patients + contraindications table

📊 Loaded 5 patients:
   1. عبد الله يوسف — 56 سنة — ذكر
   2. أحمد محمد علي — 58 سنة — ذكر
   3. سارة خالد — 34 سنة — أنثى
   4. محمود عبد الرحمن — 41 سنة — ذكر
   5. فاطمة حسن — 31 سنة — أنثى

🔍 سجل المريضة المحورية (سارة خالد):
   Diseases: ['Myasthenia Gravis']
   Abnormal labs: ['Anti-AChR Antibodies']


## 🧠 Cell 3: Load MedGemma

In [16]:
import os
# Set to 'false' when running on Kaggle/Colab with GPU
# Set to 'true' for local testing without GPU
os.environ['MEDGEMMA_MOCK'] = 'true'  # Change to 'false' for real MedGemma

from ai.medgemma_client import load_medgemma
model, tokenizer = load_medgemma()

# Quick test
from ai.medgemma_client import ask_medgemma
from ai.prompts import SYSTEM_PROMPT
response = ask_medgemma("ملخص سريع عن مريضة سارة خالد Myasthenia Gravis", system_prompt=SYSTEM_PROMPT)
print(response[:200] + "...")

🧪 Running in MOCK mode — MedGemma responses will be simulated
   Set MEDGEMMA_MOCK=false to use real model (requires GPU)
📋 ملخص الحالة:
- مريضة 34 سنة تعاني من Myasthenia Gravis مشخّصة منذ 2022
- تتناول Pyridostigmine 60mg ثلاث مرات يومياً
- سابقة دخول طوارئ بسبب تفاقم MG (يناير 2025) بعد إيقاف الدواء

⚠️ تنبيهات مهمة:
...


## 🚪 Cell 4: Reception Screen (Gradio)

In [11]:
from ui.components import CUSTOM_CSS, get_gradio_theme
from ui.reception_ui import create_reception_ui
import gradio as gr

with gr.Blocks(theme=get_gradio_theme(), css=CUSTOM_CSS, title="الاستقبال") as reception_demo:
    create_reception_ui()

reception_demo.launch(share=False, inline=True, height=800)

C:\Users\elnamaky\AppData\Local\Temp\ipykernel_3272\3969045270.py:5: UserWarning: The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: theme, css. Please pass these parameters to launch() instead.
  with gr.Blocks(theme=get_gradio_theme(), css=CUSTOM_CSS, title="الاستقبال") as reception_demo:


* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


## 🚨 Cell 5: Emergency Screen (Gradio)

In [12]:
from ui.emergency_ui import create_emergency_ui

with gr.Blocks(theme=get_gradio_theme(), css=CUSTOM_CSS, title="الطوارئ") as emergency_demo:
    create_emergency_ui()

emergency_demo.launch(share=False, inline=True, height=900)

C:\Users\elnamaky\AppData\Local\Temp\ipykernel_3272\687253237.py:3: UserWarning: The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: theme, css. Please pass these parameters to launch() instead.
  with gr.Blocks(theme=get_gradio_theme(), css=CUSTOM_CSS, title="الطوارئ") as emergency_demo:


* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


## 🔍 Cell 6: Deep Diagnosis Screen (Gradio)

In [13]:
from ui.diagnosis_ui import create_diagnosis_ui

with gr.Blocks(theme=get_gradio_theme(), css=CUSTOM_CSS, title="التشخيص المعمق") as diagnosis_demo:
    create_diagnosis_ui()

diagnosis_demo.launch(share=False, inline=True, height=800)

C:\Users\elnamaky\AppData\Local\Temp\ipykernel_3272\3336248067.py:3: UserWarning: The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: theme, css. Please pass these parameters to launch() instead.
  with gr.Blocks(theme=get_gradio_theme(), css=CUSTOM_CSS, title="التشخيص المعمق") as diagnosis_demo:


* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


## 📊 Cell 7: Demo Scenario — Sarah (Myasthenia Gravis + Magnesium)

This cell demonstrates the key scenario programmatically:
1. Load Sarah's record
2. Check magnesium against her contraindications → CRITICAL alert
3. Run the diagnostic detective loop

In [14]:
from ai.session_cache import SessionCache
from ai.analyzer import run_diagnosis_loop, check_vitals

print("=" * 60)
print("🎬 Demo Scenario: Sarah Khaled — Myasthenia Gravis Crisis")
print("=" * 60)

# Step 1: Create session cache for Sarah
cache = SessionCache(3)  # Sarah's patient_id

# Step 2: Check magnesium
print("\n🔍 Checking 'مغنيسيوم' (Magnesium)...")
alerts = cache.check_substance('Magnesium')
for alert in alerts:
    print(f"\n  {'🔴' if alert['risk_level'] == 'critical' else '🟡'} {alert['title']}")
    print(f"     {alert['message']}")
    print(f"     {alert['details']}")

# Step 3: Run diagnosis loop
print("\n" + "=" * 60)
print("🔎 Running Diagnostic Detective Loop...")
print("=" * 60)

result, sub_alerts = run_diagnosis_loop(
    cache,
    chief_complaint="ضعف عام شديد + صعوبة في البلع منذ يومين",
    form_data="الشكوى: ضعف عام مع صعوبة بلع\nالعلمات الحيوية: SpO2 93%, HR 95",
    transcript="المريضة: تناولت مغنيسيوم بجرعة عالية أمس لأن صديقتي نصحتني به\nالطبيب: هل أوقفتِ دواء Pyridostigmine؟\nالمريضة: قللت الجرعة"
)

print(result)

if sub_alerts:
    print("\n🔴 Substance Alerts Detected:")
    for a in sub_alerts:
        print(f"  {a['title']}")

🎬 Demo Scenario: Sarah Khaled — Myasthenia Gravis Crisis
✅ Session cache created for patient: سارة خالد
   Diseases: 1 | Allergies: 0 | Medications: 1 | Contraindications: 7

🔍 Checking 'مغنيسيوم' (Magnesium)...

  🔴 خطر حرج: تعارض Magnesium مع Myasthenia Gravis
     يثبط النقل العصبي العضلي (NMJ) ويزيد ضعف العضلات بشكل خطير — قد يسبب أزمة تنفسية
     المصدر: MG Foundation / UpToDate | مستوى الخطر: critical

🔎 Running Diagnostic Detective Loop...
[خطوة 1]: جمع البيانات من السجل الطبي...
   ← المريضة سارة خالد، 34 سنة
   ← مصابة بـ Myasthenia Gravis منذ 2022
   ← تتناول Pyridostigmine 60mg × 3 يومياً
   ← سابقة تفاقم MG في يناير 2025 بعد إيقاف الدواء

[خطوة 2]: تحليل العلاقة بين الشكوى والتاريخ...
   ← الشكوى "ضعف عام + صعوبة بلع" متوافقة مع تفاقم MG
   ← Anti-AChR Antibodies سابقاً: 15.2 nmol/L (إيجابية قوية)
   ← احتمال: أزمة Myasthenic Crisis (ثقة مبدئية: 65%)

[خطوة 3]: البحث عن أدلة إضافية...
   ← من المحادثة/المدخلات: المريضة تناولت مغنيسيوم
   ← 🔴 تطابق مع جدول contraindications: